<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://user-images.githubusercontent.com/16768318/73986808-75b3ca00-4936-11ea-90f1-3a6c352766ce.png" width=10% >
<img align="right" style="padding-left:10px;" src="https://user-images.githubusercontent.com/16768318/73986811-764c6080-4936-11ea-9653-a3eacc47caed.png" width=10% >

**Bienvenidos!** Este *colab notebook* es parte del curso [**Introduccion a Google Earth Engine con Python**](https://github.com/csaybar/EarthEngineMasterGIS) desarrollado por el equipo [**MasterGIS**](https://www.mastergis.com/). Obten mas informacion del curso en este [**enlace**](https://www.mastergis.com/product/google-earth-engine/). El contenido del curso esta disponible en [**GitHub**](https://github.com/csaybar/EarthEngineMasterGIS) bajo licencia [**MIT**](https://opensource.org/licenses/MIT).

### **Ejercicio Resuelto:  Mapa de Índice espectral - NDVI **


 <img src="https://user-images.githubusercontent.com/60658810/112754764-28177800-8fa3-11eb-8952-a6aa210c259f.JPG" >


In [ ]:
#Autenticar e iniciar Google Earth Engine
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=630O00J3p2DRDvZAd6XCEvN7w3S4w_fKqorXoEKx1FU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7ZL1D0YyEh1fxEn9TtcIEBe_Bhb2d6M-zefkkqCp87eZwCjE1SEpM

Successfully saved authorization token.


In [ ]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

### **1. Cargar datos vectoriales**

In [ ]:
colombia = ee.FeatureCollection('users/sergioingeo/Colombia/Col')
colombia_img = colombia.draw(color = "000000", strokeWidth = 3, pointRadius = 3)
centroide = colombia.geometry().centroid().getInfo()['coordinates']
# Mostrar el ROI
mapdisplay(centroide,{'colombia':colombia_img.getMapId()},zoom_start= 6)

### **2. Cargar datos raster (Imagenes)**

In [ ]:
#Colección de imagenes Landsat 8 "Surface Reflectance"
coleccion = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
  .filterDate('2018-01-01', '2018-12-31')\
  .filterBounds(colombia)\
  .filterMetadata('CLOUD_COVER' ,'less_than',50)


### **3. Calculo del índice normalizado**
Utiliza .normalizedDifference para realizar este ejercicio

In [ ]:
def ndvi(image):
    return image.normalizedDifference([ 'B5' ,'B4']).rename('NDVI')
ndvi = coleccion.map(ndvi).mean().clip(colombia)

palette = [
   'FFFFFF','CE7E45','DF923D','F18555','FCD163','998718',
    '74A901','66A000','529400','3E8601','207401','056201',
	'004C00','023801','012E01','011D01','011D01','011301']


NDVI= ndvi.getMapId({'min': 0, 'max': 1, 'palette':palette })
mapdisplay(centroide,{'NDVI':NDVI },zoom_start= 6)

### **4. Descargar los resultados (De Google Earth Engine a Google Drive)**
**ee.batch.Export.table.toDrive():** Guarda FeatureCollection como shapefile en Google Drive.

**ee.batch.Export.image.toDrive():** Guarda Images como GeoTIFF en Google Drive.

In [ ]:
# Donde
# image: Es la imagén raster con la informacion del índice
# description: es el nombre que tendra el archivo en Google Drive.
# folder: es la carpeta que se creara en Google Drive.
# region: es el área que se exportará del producto creado.
# maxPixels: Aumenta o limita la cantidad máxima de pixels que pueden ser exportados.
# scale: El tamaño de los pixels de la imagén que serán exportados en metros.  
task = ee.batch.Export.image.toDrive(
    image= ndvi,
    description='NDVI_Colombia',
    folder='TareaMASTERGIS',
    scale= 1000,
    region = colombia.geometry(),
    maxPixels = 1e13)
task.start()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Mensage Final del curso
%%html
<marquee style='width: 30%; color: blue;'><b>MUCHAS GRACIAS ESPERO TE HAYAS DIVERTIDO TOMANDO ESTE CURSO :3 ... HASTA UNA PROXIMA OPORTUNIDAD</b></marquee>

### **¿Dudas con este Jupyer-Notebook?**

Estaremos felices de ayudarte!. Create una cuenta Github si es que no la tienes, luego detalla tu problema ampliamente en: https://github.com/csaybar/EarthEngineMasterGIS/issues

**Tienes que dar clic en el boton verde!**

<center>
<img src="https://user-images.githubusercontent.com/16768318/79680748-d5511000-81d8-11ea-9f89-44bd010adf69.png" width = 70%>
</center>